![tenor](pictures/tenor.gif)


# Image Evaluation in Medical Clinic Diagnosis

<p style="background:black">
<code style="background:black;color:white">
The use of AI in medical treatment is becoming a powerful weapon for epidemic prevention and control. In the past, doctors who need long-term training can do professional diagnosis. Now, through the latest artificial intelligence identification system, we can diagnose the CT image of patients, and analyze the human cells infected by pathogens, so that in hospitals where medical experts and equipment are insufficient, we can also diagnose the epidemic situation in an emergency. First of all, AI is used to do basic screening and judgment, and then doctors can do control and correction, to guarantee life safety for the patients and improve the diagnosis efficiency of doctors.
  
</code>
</p>

## Images discrimination and generation by GAN


* Generative Adversarial Network (GAN) is one of the most popular Machine Learning methods developed in recent years. At a high level, GAN is composed with two neural networks: the generator and the discriminator. A Generator (“the artist”) learns to create images that look real, while a Discriminator (“the art critic”) learns to tell real images apart from fakes.
* During the training, the generator progressively becomes better at creating images that look real, while the discriminator becomes better at telling them apart. The process reaches equilibrium when the discriminator can no longer distinguish real images from fakes.



![Gan1](pictures/Gan1.png)


## Double structures of GAN model

* The structure of GAN mainly includes a Generator G and a Discriminator D. The purpose of the generator G during the training process is to learn the real data distribution to make the generated pictures more reliable, and thus "deceived" the discriminator. The goal of discriminator D is to separate the pictures generated by G from the real pictures as much as possible. In this way, G and D constitute a dynamic "game process". The whole process is that the pictures generated by the generator are more and more real, and the discriminator's performance in discriminating real pictures is more accurate. As time goes by, the two models reach an ideal balance, and G can generate "fake and untrue" pictures. For D, it is difficult to determine whether the picture generated by G is a real picture.
* Take the GAN network to generate handwritten numbers as an example, initially, the generator accepts a random noise Z, through this noise to generate a digital image, and then the discriminator is to determine whether the generated image or the real image, in the early training, the generator generated samples that are easily discarded by the discriminator, as the number of iterations increases, the generator will generate more and more similar to human handwritten numbers, and the discriminator discriminates more accurately.
* Accordingly, let’s say we want to generate the CT image that would pass the discriminator as authentic. First, we have to ask the generator to produce fake images of the MRI image. Then we have to pass both the fake CT image and the real CT image into the discriminator. The discriminator then goes to work on sorting the real from the fake. The following is the example of source code in TensorFlow. These two functions are defined as the generator and discriminator. It is important to note that both functions have two hidden layers and are connected. The first code is illustrated as the generator,


![Gan1](pictures/gan2.png)



-------
## Import the needed library

In [18]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np
  

-------
## Initinilization

In [19]:

class GAN(): 
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


-------
## The Generator part in GAN
<p style="background:DodgerBlue">
<code style="background:DodgerBlue;color:white">
model.add(Dense(256, input_dim=self.latent_dim)), it means 256 neurons with input dimension in the FIRST hidden layer of generator part.
    
</code>
</p>

In [20]:
class GAN(GAN): 

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

-------
## The Discriminator part in GAN
<p style="background:DodgerBlue">
<code style="background:DodgerBlue;color:white">
model.add(Dense(512)), it means 512 neurons with input dimension in the FIRST hidden layer in discriminator part.
    
</code>
</p>

In [21]:
class GAN(GAN): 

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

-------
## The Training part in GAN

In [22]:
class GAN(GAN): 

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

In [23]:
class GAN(GAN): 

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


-------
## The execution time of GAN
<p style="background:DodgerBlue">
<code style="background:DodgerBlue;color:white">
Don't modify the epochs number, the number is concerning to the system learning/execution time.
    
</code>
</p>

In [24]:
gan = GAN()
gan.train(epochs=300, batch_size=32, sample_interval=200)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

/root/anaconda2/envs/tensorflow/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.373048, acc.: 71.88%] [G loss: 0.458716]
2 [D loss: 0.383365, acc.: 67.19%] [G loss: 0.576796]
3 [D loss: 0.352552, acc.: 76.56%] [G loss: 0.647508]
4 [D loss: 0.359930, acc.: 81.25%] [G loss: 0.757542]
5 [D loss: 0.322538, acc.: 84.38%] [G loss: 0.872035]
6 [D loss: 0.254908, acc.: 96.88%] [G loss: 1.074164]
7 [D loss: 0.219347, acc.: 100.00%] [G loss: 1.252729]
8 [D loss: 0.181047, acc.: 100.00%] [G loss: 1.384701]
9 [D loss: 0.150271, acc.: 100.00%] [G loss: 1.630984]
10 [D loss: 0.127544, acc.: 100.00%] [G loss: 1.699853]
11 [D loss: 0.111583, acc.: 100.00%] [G loss: 1.900062]
12 [D loss: 0.109445, acc.: 100.00%] [G loss: 2.010497]
13 [D loss: 0.087252, acc.: 100.00%] [G loss: 2.216022]
14 [D loss: 0.067167, acc.: 100.00%] [G loss: 2.336895]
15 [D loss: 0.058170, acc.: 100.00%] [G loss: 2.489378]
16 [D loss: 0.062947, acc.: 100.00%] [G loss: 2.445242]
17 [D loss: 0.050396, acc.: 100.00%] [G loss: 2.604207]
18 [D loss: 0.046344, acc.: 100.00%] [G loss: 2.637858]
19 [D l